In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import pickle

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
data = pd.read_csv(f'D:\\庫存健診開發\\data\\Training\\Raw\\large.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3,cluster
0,2007-07-02,1452,宏益,4.989023,5.107506,4.894894,5.012042,4151.509770,42638.633348,12.7,...,NaN,NaN,NaN,NaN,0.212611,0.212611,0.000000,0.000000,0.000000,0
1,2007-07-03,1452,宏益,5.029036,5.149907,4.932688,5.050268,2638.807234,26221.785384,12.8,...,NaN,NaN,20.126073,NaN,0.217219,0.215069,0.000000,0.000000,0.000000,0
2,2007-07-04,1452,宏益,5.069553,5.192840,4.970949,5.088969,5101.124000,50104.353560,13.6,...,NaN,NaN,39.090848,59.216921,0.221892,0.217652,0.000000,0.000000,0.000000,0
3,2007-07-05,1452,宏益,5.110421,5.236142,5.009529,5.127996,5876.123440,56320.076781,13.5,...,NaN,NaN,45.063651,84.154499,0.226613,0.220359,0.000000,0.000000,0.000000,0
4,2007-07-06,1452,宏益,5.153602,5.281898,5.050302,5.169242,4361.808914,39842.656833,13.2,...,NaN,NaN,35.082897,80.146548,0.231595,0.223243,0.000000,0.708902,0.365885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398478,2019-09-17,9958,世紀鋼,76.599123,77.473952,75.712311,76.193901,4318.768378,339531.271421,161.8,...,0.659924,-0.142467,-4.240093,6.017040,1.761640,2.045825,0.830230,0.424065,0.506313,4
1398479,2019-09-18,9958,世紀鋼,76.539468,77.265808,75.649237,75.968260,3068.071378,235454.760807,162.6,...,0.577861,-0.270728,-9.085778,-13.325871,1.616571,1.992168,0.811675,0.321372,0.413843,4
1398480,2019-09-19,9958,世紀鋼,76.571516,77.196330,75.687225,75.893182,2462.163409,185428.207154,159.7,...,0.670798,-0.297717,-2.433332,-11.519110,1.509105,1.931785,0.811675,0.243049,0.328446,4
1398481,2019-09-20,9958,世紀鋼,76.671053,77.246500,75.801507,75.950037,2299.101425,176377.424777,158.6,...,0.612239,-0.238165,1.722372,-0.710960,1.444993,1.870936,0.815948,0.160086,0.244266,4


In [3]:
train_df, val_df, test_df = func.TrainTestSplit(data, date(2016,1,1), date(2017,9,1))
del data

D:\庫存健診開發\code\TrainingPrep\Training_function.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_df = data[data['ts'].dt.date < TrainDate][data['ema26'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_df = data[data['ts'].dt.date < valDate][data['ts'].dt.date >= TrainDate][data['ema26'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = data[data['ts'].dt.date >= valDate][data['ema26'].notnull()]


In [4]:
feature_list = ['open', 'high', 'low', 'close', 'vol',
       'VWAP', 'index_open', 'index_high', 'index_low', 'index_close', 'index_vol',
       'industry_open', 'industry_high', 'industry_low', 'industry_close',
       'industry_vol', 'foreign_buy', 'investment_buy', 'dealer_buy',
       'ma7', 'ma21', 'ema26', 'ema12', 'MACD', 'upper_band',
       'lower_band', 'index_close_corr', 'industry_close_corr',
       'foreign_buy_corr', 'investment_buy_corr', 'dealer_buy_corr',
       'close_skew', 'close_kurtosis', 'index_close_skew',
       'index_close_kurtosis', 'industry_close_skew',
       'industry_close_kurtosis', 'foreign_buy_skew', 'foreign_buy_kurtosis',
       'investment_buy_skew', 'investment_buy_kurtosis', 'dealer_buy_skew',
       'dealer_buy_kurtosis','pvt', 'ATR', 'RSI_15', 'SO5', 'SO3']

print(len(feature_list))
print(train_df[feature_list].isna().values.any())
print(val_df[feature_list].isna().values.any())
print(test_df[feature_list].isna().values.any())

48
False
False
False


In [5]:
param = func.GetScalerParam(train_df, feature_list)
param

,max,min,std,mean
open,3.353743e+03,1.483361e+00,7.222199e+01,3.477140e+01
high,3.389055e+03,1.507417e+00,7.331717e+01,3.526385e+01
low,3.292900e+03,1.456674e+00,7.101749e+01,3.424486e+01
close,3.342729e+03,1.491662e+00,7.205638e+01,3.469904e+01
vol,4.569573e+05,-1.257470e+05,1.075355e+04,5.261669e+03
VWAP,3.625711e+03,0.000000e+00,8.528927e+01,4.664470e+01
index_open,9.959143e+03,4.031350e+03,1.125844e+03,7.996162e+03
index_high,1.000754e+04,4.167043e+03,1.120658e+03,8.034198e+03
index_low,9.905978e+03,4.030078e+03,1.128541e+03,7.943120e+03
index_close,9.954860e+03,4.125144e+03,1.123112e+03,7.986787e+03


In [6]:
train_df, val_df, test_df = func.Scaler(train_df, feature_list, param), func.Scaler(val_df, feature_list, param), func.Scaler(test_df, feature_list, param)
display(train_df)
display(test_df)

,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3,cluster
24,2007-08-03,1452,宏益,0.001183,0.001201,0.001173,0.001177,0.217460,8970.226132,14.9,...,0.434884,0.174963,0.488661,0.356063,0.255779,0.001445,0.949970,0.499628,0.507462,0
25,2007-08-06,1452,宏益,0.001185,0.001203,0.001175,0.001178,0.218669,16941.193204,14.8,...,0.455061,0.164069,1.529319,0.356156,0.255823,0.001446,0.954799,0.499954,0.507828,0
26,2007-08-07,1452,宏益,0.001187,0.001204,0.001177,0.001180,0.220624,30877.883080,15.8,...,0.459115,0.166222,2.179228,0.356268,0.255754,0.001446,0.958770,0.500210,0.508227,0
27,2007-08-08,1452,宏益,0.001189,0.001205,0.001178,0.001181,0.222314,44159.614387,16.4,...,0.470232,0.150726,2.212560,0.356314,0.255536,0.001447,0.961824,0.500095,0.508330,0
28,2007-08-09,1452,宏益,0.001191,0.001207,0.001180,0.001182,0.221396,38044.934885,16.4,...,0.451233,0.160097,3.129896,0.356377,0.255555,0.001447,0.966168,0.500095,0.508382,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124418,2015-12-25,9938,百和,0.018926,0.018917,0.018908,0.018869,0.218197,92647.922011,222.3,...,0.598939,0.267158,1.770895,0.356740,1.896895,0.013259,0.555489,0.508250,0.526813,4
1124419,2015-12-28,9938,百和,0.019024,0.018962,0.018932,0.018891,0.218683,111110.052284,219.0,...,0.602970,0.262893,1.906329,0.356266,1.971022,0.013060,0.579361,0.503428,0.520453,4
1124420,2015-12-29,9938,百和,0.019048,0.018991,0.018999,0.018921,0.218667,110950.917869,215.2,...,0.599307,0.261660,2.579307,0.356320,1.850913,0.012796,0.595699,0.503238,0.517111,4
1124421,2015-12-30,9938,百和,0.019011,0.018977,0.019001,0.018906,0.219485,147223.522653,216.9,...,0.603833,0.262989,-1.583712,0.356088,1.797110,0.012526,0.595699,0.500616,0.513214,4


,ts,StockNo,StockName,open,high,low,close,vol,total,capital,...,dealer_buy_skew,dealer_buy_kurtosis,pvt_current,pvt,TR,ATR,RSI_15,SO5,SO3,cluster
1124829,2017-09-01,1439,中和,0.005793,0.005790,0.005834,0.005779,0.215908,1352.054607,19.3,...,1.158718e-01,0.655258,-1.574823,0.355917,0.669995,0.003212,0.178111,0.488849,0.504231,0
1124830,2017-09-04,1439,中和,0.005771,0.005769,0.005816,0.005762,0.215891,1069.101150,19.3,...,1.158718e-01,0.655258,-0.146112,0.355907,0.449872,0.003136,0.159799,0.488591,0.496565,0
1124831,2017-09-05,1439,中和,0.005748,0.005747,0.005798,0.005744,0.215912,1320.055131,19.1,...,1.158718e-01,0.655258,-0.185442,0.356000,0.437559,0.003061,0.142460,0.488375,0.492549,0
1124832,2017-09-06,1439,中和,0.005724,0.005725,0.005779,0.005727,0.215953,1853.937872,19.0,...,3.226233e-16,1.000000,-0.254700,0.355992,0.424291,0.002985,0.126491,0.488203,0.490394,0
1124833,2017-09-07,1439,中和,0.005692,0.005695,0.005753,0.005703,0.215974,2210.143405,18.6,...,3.226233e-16,1.000000,-0.397286,0.355978,0.406904,0.002906,0.107912,0.491186,0.491850,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398478,2019-09-17,9958,世紀鋼,0.022408,0.022425,0.022560,0.022358,0.223211,339531.271421,161.8,...,5.799039e-01,0.158289,-4.240093,0.356422,1.761640,0.012114,0.830230,0.499507,0.512879,4
1398479,2019-09-18,9958,世紀鋼,0.022390,0.022364,0.022541,0.022290,0.221064,235454.760807,162.6,...,5.699677e-01,0.151184,-9.085778,0.355136,1.616571,0.011794,0.811675,0.495503,0.506757,4
1398480,2019-09-19,9958,世紀鋼,0.022399,0.022343,0.022553,0.022268,0.220024,185428.207154,159.7,...,5.812206e-01,0.149689,-2.433332,0.355256,1.509105,0.011434,0.811675,0.492449,0.501104,4
1398481,2019-09-20,9958,世紀鋼,0.022429,0.022358,0.022587,0.022285,0.219745,176377.424777,158.6,...,5.741303e-01,0.152988,1.722372,0.355974,1.444993,0.011072,0.815948,0.489214,0.495531,4


In [7]:
df_list = [group[1] for group in train_df.groupby(train_df['StockNo'])]

X_train, Y_train = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=20, forward=5, feature_list=feature_list), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_train.append(xx.tolist())
            Y_train.extend(x[1])
            

assert len(X_train) == len(Y_train)
save_list = [X_train, Y_train]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_train', 'wb') as fp:
    pickle.dump(save_list, fp)

In [7]:
df_list = [group[1] for group in val_df.groupby(val_df['StockNo'])]

X_val, Y_val = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=20, forward=5, feature_list=feature_list), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_val.append(xx.tolist())
            Y_val.extend(x[1])

assert len(X_val) == len(Y_val)
save_list = [X_val, Y_val]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_val', 'wb') as fp:
    pickle.dump(save_list, fp)

In [8]:
df_list = [group[1] for group in test_df.groupby(test_df['StockNo'])]

X_test, Y_test = [], []

if __name__ == '__main__':
    with Pool(processes=10) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=20, forward=5, feature_list=feature_list, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_test.append(xx.tolist())
            Y_test.extend(x[1])

assert len(X_test) == len(Y_test)
save_list = [X_test, Y_test]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_test', 'wb') as fp:
    pickle.dump(save_list, fp)

In [7]:
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_train', 'rb') as fp:
    load_list = pickle.load(fp)
train_data = func.transform_whole_csv(load_list[0], load_list[1], feature_list)
train_data = pd.merge(train_data, train_df[['ts', 'StockNo', 'cluster']], on=['ts', 'StockNo'], how='left')
train_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.002263,0.002303,0.002237,0.002256,0.225242,0.003017,0.864141,0.856053,0.865558,0.860005,...,0.143095,0.355717,0.002299,0.027467,0.493005,0.495476,0,2007-09-06,1218,1
1,0.002259,0.002298,0.002232,0.002250,0.224382,0.003020,0.821859,0.825992,0.824660,0.822716,...,0.156124,0.355726,0.002251,0.027467,0.493798,0.496609,0,2007-09-07,1218,1
2,0.002253,0.002292,0.002225,0.002243,0.225996,0.003019,0.854950,0.844325,0.834970,0.831907,...,0.161973,0.356036,0.002207,0.073887,0.495221,0.498383,1,2007-09-10,1218,1
3,0.002245,0.002284,0.002218,0.002234,0.228876,0.003013,0.834789,0.848107,0.835675,0.852209,...,0.161973,0.356048,0.002168,0.112683,0.496594,0.500435,1,2007-09-11,1218,1
4,0.002240,0.002278,0.002212,0.002227,0.230907,0.003015,0.871258,0.863842,0.865963,0.865970,...,0.563712,0.356112,0.002133,0.148367,0.497138,0.501923,1,2007-09-12,1218,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888054,0.013860,0.013885,0.013982,0.013887,0.224440,0.011935,0.751128,0.742744,0.749802,0.747489,...,0.731551,0.354341,0.005365,0.395054,0.490866,0.493776,0,2015-12-24,9945,3
888055,0.013936,0.013965,0.014055,0.013961,0.217818,0.011993,0.733688,0.731544,0.733992,0.733117,...,0.736400,0.354158,0.005244,0.311885,0.490648,0.493084,1,2015-12-25,9945,3
888056,0.013999,0.014031,0.014117,0.014022,0.216852,0.012041,0.729041,0.725541,0.727577,0.723811,...,0.735977,0.354111,0.005147,0.244178,0.490608,0.492704,1,2015-12-28,9945,3
888057,0.014048,0.014081,0.014165,0.014070,0.220787,0.012079,0.730370,0.719938,0.723249,0.717575,...,0.735633,0.353953,0.005073,0.191684,0.490535,0.492452,1,2015-12-29,9945,3


In [8]:
train_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\train_large.csv', index=False)

In [10]:
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_val', 'rb') as fp:
    load_list = pickle.load(fp)
val_data = func.transform_whole_csv(load_list[0], load_list[1], feature_list)
val_data= pd.merge(val_data, val_df[['ts', 'StockNo', 'cluster']], on = ['ts', 'StockNo'], how='left')
val_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.004561,0.004552,0.004589,0.004562,0.218600,0.004739,0.707555,0.696303,0.694806,0.688976,...,0.781089,0.356498,0.002038,0.917818,0.509096,0.523106,1,2016-02-15,1201,2
1,0.004549,0.004538,0.004579,0.004550,0.217029,0.004727,0.677415,0.673807,0.677543,0.671906,...,0.689513,0.356664,0.002090,0.933614,0.509410,0.523678,0,2016-02-16,1201,2
2,0.004538,0.004525,0.004570,0.004539,0.216909,0.004717,0.665264,0.659220,0.657944,0.657388,...,0.686956,0.356768,0.002144,0.945195,0.509695,0.524205,0,2016-02-17,1201,2
3,0.004527,0.004511,0.004562,0.004527,0.216208,0.004707,0.637578,0.633498,0.623533,0.622528,...,0.763975,0.356729,0.002196,0.954566,0.510018,0.524743,1,2016-02-18,1201,2
4,0.004519,0.004502,0.004556,0.004520,0.215749,0.004700,0.625076,0.620828,0.623737,0.625763,...,0.763975,0.356861,0.002245,0.961994,0.510377,0.525317,1,2016-02-19,1201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196140,0.010532,0.010478,0.010666,0.010543,0.221141,0.009114,1.085322,1.081665,1.091045,1.087228,...,0.564662,0.356554,0.002645,0.659460,0.505172,0.514478,1,2017-08-24,9945,3
196141,0.010523,0.010468,0.010657,0.010534,0.220598,0.009103,1.082659,1.080653,1.087066,1.087647,...,0.571674,0.356651,0.002699,0.730819,0.506502,0.516895,1,2017-08-25,9945,3
196142,0.010514,0.010460,0.010648,0.010525,0.220610,0.009093,1.085993,1.082591,1.093648,1.088703,...,0.533162,0.356762,0.002761,0.785587,0.507693,0.519114,1,2017-08-28,9945,3
196143,0.010506,0.010452,0.010640,0.010517,0.220292,0.009083,1.080872,1.080214,1.086441,1.079510,...,0.402298,0.357081,0.002831,0.829245,0.508777,0.521144,1,2017-08-29,9945,3


In [11]:
val_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\val_large.csv', index=False)

In [12]:
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_Weekly_test', 'rb') as fp:
    load_list = pickle.load(fp)
test_data = func.transform_whole_csv(load_list[0], load_list[1], feature_list)
test_data= pd.merge(test_data, test_df[['ts', 'StockNo', 'cluster']], on = ['ts', 'StockNo'], how='left')
test_data

,open_day1,high_day1,low_day1,close_day1,vol_day1,VWAP_day1,index_open_day1,index_high_day1,index_low_day1,index_close_day1,...,dealer_buy_kurtosis_day20,pvt_day20,ATR_day20,RSI_15_day20,SO5_day20,SO3_day20,Y,ts,StockNo,cluster
0,0.017219,0.017194,0.017412,0.017259,0.217392,0.017993,1.105036,1.103670,1.109443,1.107818,...,0.184608,0.356233,0.006303,0.418035,0.507918,0.516443,1,2017-10-05,1215,2
1,0.017274,0.017244,0.017466,0.017310,0.217565,0.018046,1.105916,1.104205,1.109014,1.103198,...,0.191746,0.356399,0.006271,0.499704,0.510310,0.521110,1,2017-10-06,1215,2
2,0.017332,0.017299,0.017524,0.017365,0.217753,0.018103,1.107064,1.106146,1.112574,1.112918,...,0.215796,0.356472,0.006229,0.588020,0.512006,0.524884,1,2017-10-11,1215,2
3,0.017395,0.017357,0.017585,0.017425,0.217936,0.018164,1.108468,1.106521,1.110848,1.107681,...,0.186648,0.356403,0.006194,0.632409,0.512552,0.527234,1,2017-10-12,1215,2
4,0.017474,0.017432,0.017663,0.017500,0.217972,0.018241,1.107070,1.106523,1.111666,1.110279,...,0.189088,0.356323,0.006183,0.647303,0.512033,0.527969,1,2017-10-13,1215,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258852,0.021428,0.021411,0.021486,0.021335,0.218549,0.020209,1.093657,1.088233,1.091523,1.085612,...,0.350552,0.361046,0.012713,0.789602,0.514507,0.531547,1,2019-09-16,9958,4
258853,0.021453,0.021437,0.021542,0.021382,0.218316,0.020255,1.089134,1.084912,1.088800,1.083482,...,0.312014,0.360640,0.012722,0.807274,0.514063,0.531848,1,2019-09-17,9958,4
258854,0.021528,0.021520,0.021654,0.021486,0.218034,0.020357,1.067509,1.070854,1.070677,1.069737,...,0.312334,0.358430,0.012698,0.818501,0.512778,0.530909,1,2019-09-18,9958,4
258855,0.021511,0.021509,0.021659,0.021490,0.218183,0.020360,1.062800,1.067903,1.066920,1.067981,...,0.223767,0.356851,0.012638,0.818501,0.509132,0.527343,0,2019-09-19,9958,4


In [13]:
test_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\test_large.csv', index=False)